In [277]:
using LinearAlgebra

using Plots
using SparseArrays
using Pkg
using Arpack
using BlockDiagonals

In [337]:
function P(H, σ, K)
    
    function c(n, sigma)
        if n == 1
            p = sqrt(4-3)*cos((n-1)*acos(sigma))
        else
            p = 2*cos((n-1)*acos(sigma))
        end
        return p
    end
    
    function T(H, n)
        H = Matrix(H)
        TH = [H^0 for i in 1:n]
        if n > 1
            TH[2] = H 

            for i in 2:n-1
                TH[i+1] = 2*H*TH[i] - TH[i-1]
            end
        end
        return TH
    end
    
    #D = K^2 #not determin
    TH = T(H, K)
    p = 0 * H^0
    for n in 1:K
        p = p + c(n, σ)*TH[n]
    end
    return p
end

P (generic function with 1 method)

In [353]:
function D(σ, K)
    
    function c(n, sigma)
        if n == 1
            p = sqrt(4-3)*cos((n-1)*acos(sigma))
        else
            p = 2*cos((n-1)*acos(sigma))
        end
        return p
    end
    
    function T(sigma, n)
        Ts = [sigma^0 for i in 1:n]
        if n > 1
            Ts[2] = sigma

            for i in 2:n-1
                Ts[i+1] = 2*s*Ts[i] - Ts[i-1]
            end
        end
        return Ts
    end
    
    #D = K^2 #not determin
    Ts = T(σ, K)
    p = 0
    for n in 1:K
        p = p + c(n, σ)*Ts[n]
    end
    return p
end   

D (generic function with 1 method)

In [363]:
H = Matrix(Hermitian(sprand(1000,1000,0.01)))
PH = P(H,0,10)./D(0,10)

1000×1000 Matrix{Float64}:
 -0.00706184   -0.00176802   -0.00235515   …  -0.00215176   -0.00104134
 -0.00176802   -0.00352877   -0.00150554      -0.00128672   -0.00086106
 -0.00235515   -0.00150554   -0.00520033      -0.00142719   -0.000844627
 -0.00147721   -0.00109132   -0.00121962      -0.00111162   -0.000624021
 -0.0013423    -0.00116731   -0.001253        -0.00097932   -0.000997894
 -0.00363802   -0.00229795   -0.00259484   …  -0.0027546    -0.00127082
 -0.00316253   -0.00212304   -0.00240336      -0.00316885   -0.00106204
 -0.00145598   -0.00124521   -0.00141021      -0.00107618   -0.000554234
 -0.00555232   -0.00297518   -0.00377297      -0.00276217   -0.00147767
 -0.000783592  -0.000657047  -0.000814254     -0.000553062  -0.000391549
 -0.0010268    -0.00087078   -0.000924655  …  -0.00070933   -0.000391408
 -0.00269322   -0.00282779   -0.00216316      -0.00175769   -0.00100556
 -0.00187456   -0.00192188   -0.00164801      -0.00189492   -0.000744705
  ⋮                           

In [ ]:
N = 10000
s = 3
A = Matrix(sprand(N,s,0.1))
H = Matrix(Hermitian(sprand(N,N,0.01)))

Q1, R = qr(A)
Q1 = Matrix(Q1)

σ = 0
K = 10
m = 100
B1 = Matrix{Float64}(I,s,s)
A = []
B = [B1]
Q = [Q1]
p = P(H,σ,K)./D(σ,K)

j = 1
while j < m
    if j == 1
        Qj = Matrix(Q[j])
        
        Uj = p*Qj
        
        Aj = transpose(Qj)*Uj
        
        Rj1 = Uj - Qj*Aj
        
        Qj1, Bj1 = qr(Rj1)
        
        push!(B,Matrix(Bj1))
        push!(Q, Matrix(Qj1))
        push!(A,Matrix(Aj))
        j = j+1
    else
        Qj = Matrix(Q[j])
        Bj = Matrix(B[j])
        
        Uj = p*Qj - Q[j-1]*transpose(B[j])
        Aj = transpose(Qj)*Uj
        Rj1 = Uj - Qj*Aj
        Qj1, Bj1 = qr(Rj1)
        push!(B, Matrix(Bj1))
        push!(Q, Matrix(Qj1))
        j = j+1
    end
end


In [369]:
ℚ = hcat(Q[1], Q[2])
for i in 3:length(Q)
   ℚ = hcat(ℚ, Q[i])
end


In [370]:
Tm = transpose(ℚ)*P(H,σ,K)*ℚ

1000×1000 Matrix{Float64}:
  2.57788e7    2.13086e7    2.1974e7   …     -6.74348e6  -5.26128e6
  2.13086e7    1.86937e7    1.864e7          -5.6518e6   -4.49435e6
  2.1974e7     1.864e7      1.96855e7        -5.79289e6  -4.64394e6
  1.69792e7    1.44533e7    1.47886e7        -4.53807e6  -3.57871e6
 -1.62998e7   -1.37991e7   -1.41515e7         4.35209e6   3.30367e6
  1.69107e7    1.42359e7    1.47096e7  …     -4.47585e6  -3.55006e6
  1.89495e7    1.60679e7    1.64367e7        -5.06957e6  -3.9665e6
 -1.35687e7   -1.14632e7   -1.18624e7         3.56252e6   2.9057e6
  1.68313e7    1.42589e7    1.46416e7        -4.46121e6  -3.50271e6
  1.66664e7    1.42794e7    1.46421e7        -4.46539e6  -3.56188e6
  7.34474e7    6.24171e7    6.39515e7  …     -1.94674e7  -1.55115e7
  3.58123e-8  -2.36221e6   -1.0476e6          1.84822e5   2.66196e5
  3.19415e-8   2.54513e-8  -1.76229e6     18154.5         3.10434e5
  ⋮                                    ⋱                 
 -1.01556e6   -7.41829e5   -1.005

In [367]:
TT = convert.(Float32, Tm)
sparse(TT)

1000×1000 SparseMatrixCSC{Float32, Int64} with 999999 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

In [330]:
L = transpose(Q[2])*Q[2]
L = convert.(Float16, L)
sparse(L)

3×3 SparseMatrixCSC{Float16, Int64} with 3 stored entries:
 1.0   ⋅    ⋅ 
  ⋅   1.0   ⋅ 
  ⋅    ⋅   1.0

In [331]:
K = transpose(ℚ)*ℚ
K = convert.(Float16, K)
#sparse(K)

300×300 Matrix{Float16}:
  1.0        0.0        0.0       …   0.02666    0.02235    -0.0433
  0.0        1.0        0.0           0.02594   -0.01235    -0.00994
  0.0        0.0        1.0           0.010735  -0.003271   -0.005486
 -0.0        0.0       -0.0           0.002098  -0.0007343  -0.000442
 -0.0        0.0       -0.0          -0.208     -0.04538     0.242
  0.0        0.0       -0.0       …  -0.01125   -0.003265    0.00932
  0.0       -0.0       -0.0          -0.2261    -0.03903     0.1644
 -0.0       -0.0        0.0           0.3782     0.07184    -0.4373
  0.0        0.0       -0.0          -0.005836  -0.0006385   0.004913
  0.0       -0.0       -0.0           0.04767   -0.01678    -0.02713
 -0.0        0.0        0.0       …   0.2167     0.04056    -0.2372
  0.0       -0.0       -0.0          -0.11865   -0.00996     0.136
 -0.0        0.0        0.0          -0.0587    -0.04263     0.0983
  ⋮                               ⋱                         
 -0.0496    -0.00792   

In [332]:
transpose(Q[1])*P(H, σ,K)*Q[1]

LoadError: MethodError: no method matching (::Colon)(::Int64, ::Matrix{Float16})
[0mClosest candidates are:
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real at C:\Users\tjwld\AppData\Local\Programs\Julia-1.7.3\share\julia\base\range.jl:41
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Real} at C:\Users\tjwld\AppData\Local\Programs\Julia-1.7.3\share\julia\base\range.jl:10
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T at C:\Users\tjwld\AppData\Local\Programs\Julia-1.7.3\share\julia\base\range.jl:40
[0m  ...

In [333]:
ℚ

1000×300 Matrix{Float64}:
  0.0         -3.98986e-17  -2.94903e-17  …   0.00363986   -0.00251457
  0.0          0.0          -8.67362e-18     -5.56023e-9    1.57005e-9
  0.0          0.0           0.0              1.05051e-6   -8.98274e-7
  0.0          0.0           0.0             -0.000274554   0.000404221
  0.0         -0.140926      0.00807102      -0.00178412    0.00228036
  0.0          0.0           0.0          …   0.0           0.0
  0.0          0.0           0.0             -0.014908     -0.00732368
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0          -0.166053     …   0.00278215    0.000188879
  0.0          0.0           0.0              0.0           0.0
  0.0          0.0           0.0              0.0           0.0
  ⋮                                       ⋱                
  0.0          0.0           0.

In [233]:
ℚ

1000×100 Matrix{Float64}:
 0.0   4.33681e-19  -2.10032e-17   3.76639e-18  …   0.0460942    0.0288217
 0.0   0.0           2.77556e-17  -3.68629e-17     -0.0216889    0.0287225
 0.0  -0.161709     -0.00895896    0.0142646       -0.0329841    0.0192125
 0.0   0.0           0.0           0.0              0.0525045   -0.0228856
 0.0   0.0           0.0           0.0              0.00247361  -0.0250086
 0.0   0.0           0.0305174    -0.0566639    …   0.032166     0.0722569
 0.0   0.0           0.0           0.0              0.0170427    0.0154984
 0.0   0.0           0.0          -0.0697273       -0.020585    -0.0142872
 0.0   0.0           0.0           0.0             -0.0269562    0.0337781
 0.0   0.0           0.0           0.0             -0.00194328   0.00175125
 0.0   0.0           0.0           0.0          …  -0.00923009   0.00620602
 0.0   0.0           0.0           0.0              0.0100445   -0.00256988
 0.0   0.0           0.0           0.0             -0.0234987    0.0230

In [234]:
inv(ℚ)

LoadError: DimensionMismatch("matrix is not square: dimensions are (1000, 100)")

In [235]:
ℚ

1000×100 Matrix{Float64}:
 0.0   4.33681e-19  -2.10032e-17   3.76639e-18  …   0.0460942    0.0288217
 0.0   0.0           2.77556e-17  -3.68629e-17     -0.0216889    0.0287225
 0.0  -0.161709     -0.00895896    0.0142646       -0.0329841    0.0192125
 0.0   0.0           0.0           0.0              0.0525045   -0.0228856
 0.0   0.0           0.0           0.0              0.00247361  -0.0250086
 0.0   0.0           0.0305174    -0.0566639    …   0.032166     0.0722569
 0.0   0.0           0.0           0.0              0.0170427    0.0154984
 0.0   0.0           0.0          -0.0697273       -0.020585    -0.0142872
 0.0   0.0           0.0           0.0             -0.0269562    0.0337781
 0.0   0.0           0.0           0.0             -0.00194328   0.00175125
 0.0   0.0           0.0           0.0          …  -0.00923009   0.00620602
 0.0   0.0           0.0           0.0              0.0100445   -0.00256988
 0.0   0.0           0.0           0.0             -0.0234987    0.0230

In [236]:
K = ℚ[1:100,1:10]*transpose(ℚ[1:100,1:10])
K = convert.(Float16, K)
sparse(K)

100×100 SparseMatrixCSC{Float16, Int64} with 3364 stored entries:
⠿⠇⠿⠿⠇⠿⠇⠀⠀⠿⠇⠇⠀⠀⠿⠿⠀⠿⠿⠇⠸⠸⠿⠇⠿⠀⠿⠸⠸⠸⠿⠿⠿⠇⠿⠇⠸⠇⠿⠸
⣿⡇⣿⣿⡇⣿⡇⠀⠀⣿⡇⡇⠀⠀⣿⣿⠀⣿⣿⡇⢸⢸⣿⡇⣿⠀⣿⢸⢸⢸⣿⣿⣿⡇⣿⡇⢸⡇⣿⢸
⣭⡅⣭⣭⡅⣭⡅⠀⠀⣭⡅⡅⠀⠀⣭⣭⠀⣭⣭⡅⢨⢨⣭⡅⣭⠀⣭⢨⢨⢨⣭⣭⣭⡅⣭⡅⢨⡅⣭⢨
⠉⠁⠉⠉⠁⠉⠁⠀⠀⠉⠁⠁⠀⠀⠉⠉⠀⠉⠉⠁⠈⠈⠉⠁⠉⠀⠉⠈⠈⠈⠉⠉⠉⠁⠉⠁⠈⠁⠉⠈
⣤⡄⣤⣤⡄⣤⡄⠀⠀⣤⡄⡄⠀⠀⣤⣤⠀⣤⣤⡄⢠⢠⣤⡄⣤⠀⣤⢠⢠⢠⣤⣤⣤⡄⣤⡄⢠⡄⣤⢠
⠭⠅⠭⠭⠅⠭⠅⠀⠀⠭⠅⠅⠀⠀⠭⠭⠀⠭⠭⠅⠨⠨⠭⠅⠭⠀⠭⠨⠨⠨⠭⠭⠭⠅⠭⠅⠨⠅⠭⠨
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣿⡇⣿⣿⡇⣿⡇⠀⠀⣿⡇⡇⠀⠀⣿⣿⠀⣿⣿⡇⢸⢸⣿⡇⣿⠀⣿⢸⢸⢸⣿⣿⣿⡇⣿⡇⢸⡇⣿⢸
⣤⡄⣤⣤⡄⣤⡄⠀⠀⣤⡄⡄⠀⠀⣤⣤⠀⣤⣤⡄⢠⢠⣤⡄⣤⠀⣤⢠⢠⢠⣤⣤⣤⡄⣤⡄⢠⡄⣤⢠
⠿⠇⠿⠿⠇⠿⠇⠀⠀⠿⠇⠇⠀⠀⠿⠿⠀⠿⠿⠇⠸⠸⠿⠇⠿⠀⠿⠸⠸⠸⠿⠿⠿⠇⠿⠇⠸⠇⠿⠸
⣒⡂⣒⣒⡂⣒⡂⠀⠀⣒⡂⡂⠀⠀⣒⣒⠀⣒⣒⡂⢐⢐⣒⡂⣒⠀⣒⢐⢐⢐⣒⣒⣒⡂⣒⡂⢐⡂⣒⢐
⠿⠇⠿⠿⠇⠿⠇⠀⠀⠿⠇⠇⠀⠀⠿⠿⠀⠿⠿⠇⠸⠸⠿⠇⠿⠀⠿⠸⠸⠸⠿⠿⠿⠇⠿⠇⠸⠇⠿⠸
⠛⠃⠛⠛⠃⠛⠃⠀⠀⠛⠃⠃⠀⠀⠛⠛⠀⠛⠛⠃⠘⠘⠛⠃⠛⠀⠛⠘⠘⠘⠛⠛⠛⠃⠛⠃⠘⠃⠛⠘
⣛⡃⣛⣛⡃⣛⡃⠀⠀⣛⡃⡃⠀⠀⣛⣛⠀⣛⣛⡃⢘⢘⣛⡃⣛⠀⣛⢘⢘⢘⣛⣛⣛⡃⣛⡃⢘⡃⣛⢘
⣒⡂⣒⣒⡂⣒⡂⠀⠀⣒⡂⡂⠀⠀⣒⣒⠀⣒⣒⡂⢐⢐⣒⡂⣒⠀⣒⢐⢐⢐⣒⣒⣒⡂⣒⡂⢐⡂⣒⢐
⣿⡇⣿⣿⡇⣿⡇⠀⠀⣿⡇⡇⠀⠀⣿⣿⠀⣿⣿⡇⢸⢸⣿⡇⣿⠀⣿⢸⢸⢸⣿⣿⣿⡇⣿⡇⢸⡇⣿⢸
⠿⠇⠿⠿⠇⠿⠇⠀⠀⠿⠇⠇⠀⠀⠿⠿⠀⠿⠿⠇⠸⠸⠿⠇⠿⠀⠿⠸⠸⠸⠿⠿⠿⠇⠿⠇⠸⠇⠿⠸
⠿⠇⠿⠿⠇⠿⠇⠀⠀⠿⠇⠇⠀⠀⠿⠿⠀⠿⠿⠇⠸⠸⠿⠇⠿⠀⠿⠸⠸⠸⠿⠿⠿⠇⠿⠇⠸⠇⠿⠸
⠶⠆⠶⠶⠆⠶⠆⠀⠀⠶⠆⠆⠀⠀⠶⠶⠀⠶⠶⠆⠰⠰⠶⠆⠶⠀⠶⠰⠰⠰⠶⠶⠶⠆⠶⠆⠰⠆⠶⠰
⣛⡃⣛⣛⡃⣛⡃⠀⠀⣛⡃⡃⠀⠀⣛⣛⠀⣛⣛⡃⢘⢘⣛⡃⣛⠀⣛⢘⢘⢘⣛⣛⣛⡃⣛⡃⢘⡃⣛⢘

In [237]:
ℚ*transpose(ℚ)

1000×1000 Matrix{Float64}:
  0.391776    -0.0123154     0.00791814  …  -0.0186312     0.0153362
 -0.0123154    0.0917995    -0.00297023      0.00432733    0.00131804
  0.00791814  -0.00297023    0.119464        0.0277983    -0.00128548
  0.042618     0.00368411   -0.0153725      -0.0063076    -0.00562304
 -0.00617636  -0.0254483     0.00327897     -0.00937989    0.00835705
  0.00777529   0.0084722    -0.00606063  …   0.0112888    -0.00404117
 -0.0010846   -0.0035132    -0.00123732     -0.00360785    0.00429617
  0.00218587  -0.000920643  -0.00267085     -0.00420752    0.0107652
 -0.0130414    0.0125054     0.00791893      0.0165328     0.0035433
 -0.00823213   0.0118526    -0.00213596      0.0219322     0.0196561
 -0.00277124   0.0186392     0.013198    …   0.00618479    0.0168616
  0.0306805    0.00782215    0.0219229      -0.0037889     0.00597467
 -0.0318615    0.0424237    -0.00243098     -0.00518327    0.0163014
  ⋮                                      ⋱                
  0.024305

In [238]:
K

100×100 Matrix{Float16}:
  0.003147   -0.0002266   -0.0002518   …  0.0  -0.000903  0.0   0.003468
 -0.0002266   0.00598     -0.00011903     0.0  -0.000582  0.0  -0.0004208
 -0.0002518  -0.00011903   0.02722        0.0   0.02058   0.0  -0.000777
 -0.0         0.0         -0.0            0.0   0.0       0.0  -0.0
 -0.0         0.0         -0.0            0.0   0.0       0.0  -0.0
 -0.000585    0.013596    -0.001064    …  0.0  -0.003222  0.0   0.001519
 -2.2e-5     -2.5e-6      -0.0001278      0.0   0.00041   0.0  -5.07e-5
 -0.000254    0.0056      -0.000907       0.0  -0.00216   0.0   0.002672
 -0.000552   -0.000346    -0.001916       0.0   0.01131   0.0  -0.000566
 -0.0007315  -0.000582    -0.00304        0.0   0.01136   0.0  -0.000884
 -0.0001022  -0.0001802   -0.0008254   …  0.0  -0.001299  0.0  -0.0002306
  0.0         0.0          0.0            0.0   0.0       0.0   0.0
  0.00446    -0.0003211   -0.0003567      0.0  -0.001279  0.0   0.004913
  ⋮                                    ⋱